E2 Novice

Brian Bannon

Bridges are critical components of transportation infrastructure, and their safety depends on how well they resist loads without excessive bending or deflection. This report analyzes the structural behavior of a simply supported bridge subjected to a single point load. Using fundamental beam theory, Python functions were developed to calculate bending moments and vertical deflections at various locations along the bridge span.

The purpose of this analysis is to evaluate whether different bridge types are adequate for typical real-world loads, including pedestrian, vehicle, and truck loading. For each scenario, the maximum bending moment, maximum deflection, deflection ratio, and compliance with the serviceability limit of span/360 are determined. This approach demonstrates how computational tools can support engineering decision-making and help ensure public safety.

In [43]:
import numpy as np
import pandas as pd

In [36]:
def calculate_single_moment(bridge_length: float, load: float, load_position: float, x: float) -> float:
    """
    Function to calculate the internal bending moment at a specific point along a simply supported bridge due to a single point load.
    :param bridge_length: Total length of the bridge in meters.
    :param load: Magnitude of the applied point load in kilonewtons (kN).
    :param load_position: Distance from the left support to the applied load in meters.
    :param x: Position along the bridge where the bending moment is calculated in meters.
    :return: Bending moment at position x in kilonewton-meters (kN-m).
    """
    if x <= load_position:
        b = bridge_length - load_position
        return (load * b * x) / bridge_length
    else:
        return (load * load_position * (bridge_length - x)) / bridge_length


In [37]:
def calculate_deflection(bridge_length: float, load: float, load_position: float, x: float, ei: float) -> float:
    """
    Calculates the vertical deflection (sagging) at a specific point along a simply supported bridge due to a single point load.
    :param bridge_length: Total span of the bridge in meters.
    :param load: Force applied in kilonewtons (kN).
    :param load_position: Distance from the left support to the applied load in meters.
    :param x: Position along the bridge to calculate deflection in meters.
    :param ei: Flexural rigidity (stiffness) in kN-m² (product of Elastic Modulus and Moment of Inertia).
    :return: Deflection at position x in meters (float, positive = downward).
    """
    l = bridge_length
    p = load
    a = load_position
    b = l - a

    if x <= a:
        deflection = (p * b * x) / (6 * l * ei) * (l ** 2 - b ** 2 - x ** 2)
    else:
        deflection = (p * a) / (6 * l * ei) * ((l - x) * (2 * l * x - x ** 2 - a ** 2))

    return deflection

In [38]:
def create_analysis_summary(bridge_specs: dict, load_specs: dict) -> dict:
    """
    Creates a summary of the structural analysis for a simply supported bridge under a single point load.
    :param bridge_specs: dictionary with 3 keys: length, EI and name
    :param load_specs: dictionary with 3 keys: magnitude, position and type
    :return: a dictionary with 6 keys: max_moment, max_moment_location, max_deflection, max_deflection_location, deflection_ratio and status
    """
    l = bridge_specs['length']
    ei = bridge_specs['EI']

    p = load_specs['magnitude']
    a = load_specs['position']

    test_points = np.linspace(0, l, 101)
    moments = [calculate_single_moment(l, p, a, x) for x in test_points]
    max_moment = max(moments)
    moment_index = moments.index(max_moment)
    max_moment_location = test_points[moment_index]

    deflections = [calculate_deflection(l, p, a, x, ei) for x in test_points]
    max_deflection = max(deflections)
    deflection_index = deflections.index(max_deflection)
    max_deflection_location = test_points[deflection_index]

    if max_deflection > 0:
        deflection_ratio = l / max_deflection
    else:
        deflection_ratio = float('inf')

    status = 'PASS' if max_deflection <= l / 360 else 'FAIL'

    return {
        'max_moment': max_moment,
        'max_moment_location': max_moment_location,
        'max_deflection': max_deflection,
        'max_deflection_location': max_deflection_location,
        'deflection_ratio': deflection_ratio,
        'status': status
    }

In [39]:
def display_data(data_dict: dict):
    """
    Function to make the data into a dataframe and return it
    :param data_dict: the data to be converted
    :return: a dataframe with the data
    """
    df: pd.DataFrame = pd.DataFrame(data_dict)
    bridge_specs: dict = {'length': df['Value'][1], 'EI': df['Value'][3], 'name': df['Value'][0]}
    load_specs: dict = {'magnitude': df['Value'][4], 'position': df['Value'][5], 'type': df['Value'][6]}
    summary: dict = create_analysis_summary(bridge_specs, load_specs)
    df.loc[len(df)] = ('Maximum bending moment (kN-m)', summary['max_moment'])
    df.loc[len(df)] = ('Maximum bending moment location (m)', summary['max_moment_location'])
    df.loc[len(df)] = ('Maximum deflection (mm)', summary['max_deflection'] * 1000)
    df.loc[len(df)] = ('Maximum deflection location (m)', summary['max_deflection_location'])
    df.loc[len(df)] = ('Deflection ratio', summary['deflection_ratio'])
    df.loc[len(df)] = ('Meets requirements', summary['status'])
    return df

In [40]:
data: dict = {
    'Parameter': ('Name', 'Length (m)', 'Type', 'EI (kN-m²)', 'Load magnitude (kN)', 'Load position (m)', 'Load type'),
    'Value': ('Pedestrian Footbridge in City Park', 10, 'Timber footbridge', 5000, 0.8, 5, 'Pedestrian')
}
display_data(data)

,Parameter,Value
0,Name,Pedestrian Footbridge in City Park
1,Length (m),10
2,Type,Timber footbridge
3,EI (kN-m²),5000
4,Load magnitude (kN),0.8
5,Load position (m),5
6,Load type,Pedestrian
7,Maximum bending moment (kN-m),2.0
8,Maximum bending moment location (m),5.0
9,Maximum deflection (mm),3.333333


The pedestrian footbridge is more than adequate for the applied pedestrian load. The maximum deflection is approximately 3.33 mm, which is far below the allowable deflection limit of span/360 (27.8 mm). The deflection ratio of 3000 indicates a very stiff structure relative to its span. The bending moment is also small, confirming that the bridge experiences minimal structural demand under a single pedestrian. Overall, this bridge has significant reserve capacity and comfortably meets serviceability requirements for pedestrian use.

In [41]:
data: dict = {
    'Parameter': ('Name', 'Length (m)', 'Type', 'EI (kN-m²)', 'Load magnitude (kN)', 'Load position (m)', 'Load type'),
    'Value': ('Residential Street Bridge', 15, 'Light concrete bridge', 25000, 15, 6, 'Vehicle')
}
display_data(data)

,Parameter,Value
0,Name,Residential Street Bridge
1,Length (m),15
2,Type,Light concrete bridge
3,EI (kN-m²),25000
4,Load magnitude (kN),15
5,Load position (m),6
6,Load type,Vehicle
7,Maximum bending moment (kN-m),54.0
8,Maximum bending moment location (m),6.0
9,Maximum deflection (mm),40.003605


The residential street bridge is adequate but close to the serviceability limit under the vehicle load. The maximum deflection is approximately 40.0 mm, which is slightly below the allowable deflection limit of 41.7 mm (span/360). While the bridge technically meets code requirements and passes the deflection check, the deflection ratio of approximately 375 indicates limited stiffness margin. This suggests that heavier vehicles or multiple simultaneous loads could cause excessive deflection, and additional stiffness may be desirable for long-term performance.

In [42]:
data: dict = {
    'Parameter': ('Name', 'Length (m)', 'Type', 'EI (kN-m²)', 'Load magnitude (kN)', 'Load position (m)', 'Load type'),
    'Value': ('Highway bridge', 25, 'Steel-reinforced concrete', 150000, 50, 10, 'Truck')
}
display_data(data)

,Parameter,Value
0,Name,Highway bridge
1,Length (m),25
2,Type,Steel-reinforced concrete
3,EI (kN-m²),150000
4,Load magnitude (kN),50
5,Load position (m),10
6,Load type,Truck
7,Maximum bending moment (kN-m),300.0
8,Maximum bending moment location (m),10.0
9,Maximum deflection (mm),102.889931


The highway bridge is not adequate for the applied truck load at the given position. The maximum deflection is approximately 102.9 mm, which exceeds the allowable deflection limit of 69.4 mm (span/360). The deflection ratio of approximately 243 confirms that the bridge experiences excessive sagging relative to its span. Although the bending moment is within expected ranges for a point load, the deflection failure indicates insufficient flexural rigidity for this loading condition. Structural strengthening or increased stiffness would be required for the bridge to meet serviceability criteria.

QUESTION 1

For Scenario 2, the bending moments are greatest when the load is closest to midspan. The maximum bending moment occurs at 7.5 m, because a point load produces the largest moment when it is positioned near the center of a simply supported bridge. This happens because the lever arms to both supports are most balanced at midspan.

QUESTION 2
The highway bridge (Scenario 3) has the largest absolute deflection at approximately 102.9 mm. It also has the worst deflection ratio (≈ 243), meaning the most deflection relative to its span. This occurs because the truck load is large relative to the bridge stiffness (EI), despite the bridge being designed for heavier loads.

QUESTION 3
The maximum allowable deflections are 27.8 mm (10 m bridge), 41.7 mm (15 m bridge), and 69.4 mm (25 m bridge). Scenarios 1 and 2 both meet the span/360 deflection requirement, while Scenario 3 exceeds the allowable deflection. Therefore, only the highway bridge fails the serviceability requirement.

QUESTION 4
When the truck load in Scenario 3 is moved to the center of the bridge (12.5 m), the maximum deflection increases compared to the original position at 10 m. A point load placed at midspan causes the greatest deflection in a simply supported bridge. Therefore, the center position creates more deflection than the off-center location.

QUESTION 5
Doubling the load in Scenario 1 would double the maximum bending moment from 2.0 kN·m to 4.0 kN·m. To keep deflection under the span/360 limit, the flexural rigidity (EI) would also need to double. This is because deflection is directly proportional to load and inversely proportional to EI.

In this analysis, three different bridge scenarios were evaluated under representative point loads to assess their structural performance. The pedestrian footbridge exhibited minimal bending and deflection, indicating a large margin of safety and excellent serviceability. The residential street bridge met deflection requirements but operated close to the allowable limit, suggesting limited reserve capacity under heavier or additional loads. The highway bridge failed the deflection criterion, demonstrating that its flexural rigidity is insufficient for the applied truck load at the given position.

Overall, the results highlight the importance of bridge stiffness, span length, and load placement in controlling deflection. Even when bending moments appear reasonable, excessive deflection can govern design and determine whether a bridge is acceptable. This study reinforces the need for careful structural analysis to ensure bridges meet code requirements and provide safe, reliable performance for users.